In [43]:
'''
导入包
'''
import tushare as ts
import datetime
from dateutil.relativedelta import relativedelta
import openpyxl
from decimal import Decimal

print('导入包')

导入包


In [44]:
# 全局的业务变量
class BusinessVal:
    # 标的资产，暂时只支持标的资产是一种指数
    underlyingAsset = '000001.SH'
    # 敲入比例
    knockInRatio = Decimal('0.80')
    # 敲出比例
    knockOutRatio = Decimal('1.00')
    # 期限，单位是月
    timeLimit = 12
    # start_date
    startDate = '20190101'
    # end_date
    endDate = '20200531'
    # 月利率
    monthlyRate = Decimal('0.1')
    # 保证金
    deposit = Decimal('20')
    # 名义本金倍数，名义本金除以保证金
    multiple = Decimal('5')

# 全局的技术变量
class TechVal:
    # tushare_token
    tushareToken = '19280d5f247ae49caae39e24a4363308c5396514275535ed770716c2'
    unKnowTradeDate = '无法确定交易日'

print('标的资产' + ' ' * 3 + str(BusinessVal.underlyingAsset))
print('敲出比例' + ' ' * 3 + str(BusinessVal.knockOutRatio))
print('敲入比例' + ' ' * 3 + str(BusinessVal.knockInRatio))
print('期限' + ' ' * 3 + str(BusinessVal.timeLimit))
print('模拟开始日期' + ' ' * 3 + str(BusinessVal.startDate))
print('模拟停止日期' + ' ' * 3 + str(BusinessVal.endDate))
print('月利率' + ' ' * 3 + str(BusinessVal.monthlyRate))
print('保证金' + ' ' * 3 + str(BusinessVal.deposit))
print('杠杆倍数' + ' ' * 3 + str(BusinessVal.multiple))

标的资产   000001.SH
敲出比例   1.00
敲入比例   0.80
期限   12
模拟开始日期   20100101
模拟停止日期   20200531
月利率   0.1
保证金   20
杠杆倍数   5


In [45]:
# 初始化 tushare
ts.set_token(TechVal.tushareToken)
pro = ts.pro_api()

print('初始化tushare')

初始化tushare


In [46]:
'''
获取行情数据
'''
quote = pro.index_daily(ts_code=BusinessVal.underlyingAsset, start_date = BusinessVal.startDate, end_date=BusinessVal.endDate)
# 行反转，这样第一行是最早的日期
quote = quote.reindex(index=quote.index[::-1])
print(quote)

        ts_code trade_date      close       open       high        low  \
2527  000001.SH   20100104  3243.7600  3289.7500  3295.2790  3243.3190   
2526  000001.SH   20100105  3282.1790  3254.4680  3290.5120  3221.4620   
2525  000001.SH   20100106  3254.2150  3277.5170  3295.8680  3253.0440   
2524  000001.SH   20100107  3192.7760  3253.9910  3268.8190  3176.7070   
2523  000001.SH   20100108  3195.9970  3177.2590  3198.9200  3149.0170   
...         ...        ...        ...        ...        ...        ...   
4     000001.SH   20200525  2817.9697  2816.2410  2821.4978  2802.4653   
3     000001.SH   20200526  2846.5473  2827.8967  2848.3371  2825.9010   
2     000001.SH   20200527  2836.8036  2847.3198  2848.9965  2831.9281   
1     000001.SH   20200528  2846.2217  2838.2072  2861.9241  2820.1455   
0     000001.SH   20200529  2852.3512  2835.5840  2855.3756  2829.6271   

      pre_close   change  pct_chg          vol        amount  
2527  3277.1390 -33.3790  -1.0185  109447927.0  

In [47]:
'''
定义一个函数，获取最近的一个交易日
'''
# 交易日历
tradeCalendar = pro.trade_cal(is_open='1')
print(tradeCalendar)

lastTradeData = tradeCalendar.iloc[[-1]]['cal_date'].values[0]

def getLeastTradeDate(searchData):
    '''
    获取searchData的在未来最近一个交易日
    :param searchData: 传入日期
    :return: outDate
    '''
    
    if datetime.datetime.strptime(searchData,'%Y%m%d') > datetime.datetime.strptime(lastTradeData,'%Y%m%d'):
        return TechVal.unKnowTradeDate
    
    # 如果结果是空，说明不是交易日
    while (tradeCalendar.loc[tradeCalendar['cal_date'] == searchData]).empty:
        searchData = (datetime.datetime.strptime(searchData,'%Y%m%d') + relativedelta(days=1)).strftime('%Y%m%d')
        if datetime.datetime.strptime(searchData,'%Y%m%d') > datetime.datetime.strptime(lastTradeData,'%Y%m%d'):
            return TechVal.unKnowTradeDate
    
    rntDate = searchData
    return rntDate

def isTradeDate(checkDate):
    '''
    判断checkData是否是交易日
    :param checkDate: 
    :return: 
    '''
    # 如果结果是空，说明不是交易日
    if (tradeCalendar.loc[tradeCalendar['cal_date'] == checkDate]).empty:
        return False
    else:
        return True
    

     exchange  cal_date  is_open
0         SSE  19901219        1
1         SSE  19901220        1
2         SSE  19901221        1
3         SSE  19901224        1
4         SSE  19901225        1
...       ...       ...      ...
7338      SSE  20201225        1
7339      SSE  20201228        1
7340      SSE  20201229        1
7341      SSE  20201230        1
7342      SSE  20201231        1

[7343 rows x 3 columns]


In [48]:
# 模拟
def simulate(anchorDate,anchorPrice,knockOutPrice,knockInPrice,observeList):
    '''
    模拟
    :param anchorDate: 锚定日期
    :param anchorPrice: 锚定价格
    :param knockOutPrice: 敲出价格
    :param knockInPrice: 敲入价格
    :param observeList: 观察日列表
    :return: 
    '''
    # 是否曾经敲入过
    isKnockIn = False
    # 是否曾经敲出过
    isKnockOut = False
    # 模拟日期
    simulateDate = anchorDate
    # 到期日
    endDate = observeList[-1]
    # 如果没到期
    observeIndex = 0
    # 敲入信息
    knockInList = []
    knockOutList = []
    while datetime.datetime.strptime(simulateDate,'%Y%m%d') <= datetime.datetime.strptime(endDate,'%Y%m%d'):
        # 如果不是交易日
        if not isTradeDate(checkDate=simulateDate):
            simulateDate = (datetime.datetime.strptime(simulateDate,'%Y%m%d') + relativedelta(days=1)).strftime('%Y%m%d')
            continue
        # 如果是交易日
        # 获取该日期行情数据
        simulateQuote = quote.loc[quote['trade_date'] == simulateDate]
        # 判断是否敲入
        if simulateQuote['close'].values[0] < knockInPrice:
            # 只记录首次敲入信息
            if not isKnockIn:
                knockInList.append(simulateDate + ':' + str(simulateQuote['close'].values[0]))
            isKnockIn = True
        # 判断是否是观察日
        if simulateDate in observeList:
            observeIndex = observeIndex + 1
            # 判断是否敲出
            if simulateQuote['close'].values[0] > knockOutPrice:
                # 只记录首次敲出
                if not isKnockOut:
                    knockOutList.append(simulateDate + ':' + str(simulateQuote['close'].values[0]))
                isKnockOut = True
            # 没有敲入，没有敲出
            if isKnockIn == False and (isKnockOut == False and (simulateDate == endDate)):
                payType = '没有敲入，没有敲出'
                payDate = simulateDate
                payOff = BusinessVal.deposit + observeIndex * BusinessVal.deposit * BusinessVal.monthlyRate * BusinessVal.multiple
                # return (payType,payDate,payOff)
                break
            # 没有敲入，有敲出
            if isKnockIn == False and isKnockOut == True:
                payType = '没有敲入，有敲出'
                payDate = simulateDate
                payOff = BusinessVal.deposit + observeIndex * BusinessVal.deposit * BusinessVal.monthlyRate * BusinessVal.multiple
                # return (payType,payDate,payOff)
                break
            # 有敲入，有敲出
            if isKnockIn == True and isKnockOut == True:
                payType = '有敲入，有敲出'
                payDate = simulateDate
                payOff = BusinessVal.deposit + observeIndex * BusinessVal.deposit * BusinessVal.monthlyRate * BusinessVal.multiple
                # return (payType,payDate,payOff)
                break
            # 有敲入，没有敲出
            if isKnockIn == True and (isKnockOut == False and (simulateDate == endDate)):
                payType = '有敲入，没有敲出'
                payDate = simulateDate
                # 跌幅
                dropPercent = (anchorPrice -  Decimal(simulateQuote['close'].values[0]))/anchorPrice
                payOff = BusinessVal.deposit - BusinessVal.deposit * dropPercent * BusinessVal.multiple
                # return (payType,payDate,payOff)
                break
        # 如果在前面都没有break出去
        # 如果是最后一个行情日了，就没有办法再进行模拟。
        if simulateDate == lastQuoteDate:
            if (isKnockIn == False) and (isKnockOut == False):
                payType = '没有敲入，没有敲出'
                payDate = '截止' + lastQuoteDate + '未到期'
                payOff = ''
                # return (payType,payDate,payOff)
                break
            if (isKnockIn == True) and (isKnockOut == False):
                payType = '有敲入，没有敲出'
                payDate = '截止' + lastQuoteDate + '未到期'
                payOff = ''
                # return (payType,payDate,payOff)
                break
            
        simulateDate = (datetime.datetime.strptime(simulateDate,'%Y%m%d') + relativedelta(days=1)).strftime('%Y%m%d')
    
    return (payType,payDate,payOff,isKnockIn,isKnockOut,knockInList,knockOutList)


print('定义模拟器')

定义模拟器


In [49]:
wb = openpyxl.Workbook()
wsBase = wb.active
wsBase.title = '基础信息'
wsBase.append(['标的资产',str(BusinessVal.underlyingAsset)])
wsBase.append(['敲入比例',str(BusinessVal.knockInRatio)])
wsBase.append(['敲出比例',str(BusinessVal.knockOutRatio)])
wsBase.append(['期限(月)',str(BusinessVal.timeLimit)])
wsBase.append(['模拟开始日期',str(BusinessVal.startDate)])
wsBase.append(['模拟停止日期',str(BusinessVal.endDate)])
wsBase.append(['月利率',str(BusinessVal.monthlyRate)])
wsBase.append(['保证金',str(BusinessVal.deposit)])
wsBase.append(['杠杆',str(BusinessVal.multiple)])

# 最后一个行情日
lastQuoteDate = quote.iloc[[-1]]['trade_date'].values[0]
wsBase.append(['最后一个行情日',str(lastQuoteDate)])

wsSim = wb.create_sheet(title='模拟结果')
wsSim.append(['锚定日期','锚定价格','敲入价格','敲出价格','观察日列表','观察日列表是否完整','结束类型','结束日期','本金收益和','是否有敲入','是否有敲出','首次敲入信息','首次敲出信息'])

# 遍历所有行情
for index, row in quote.iterrows():
    # 锚定日期
    anchorDate = row['trade_date']
    print(anchorDate)
    # 锚定价格
    anchorPrice = Decimal(str(row['close']))
    # 敲入价格
    knockInPrice = anchorPrice * BusinessVal.knockInRatio
    # 敲出价格
    knockOutPrice = anchorPrice * BusinessVal.knockOutRatio
    # 观察日列表
    isObserveListComplete = True
    observeList = []
    for i in range(1,BusinessVal.timeLimit + 1,1):
        observeDate = getLeastTradeDate((datetime.datetime.strptime(anchorDate,'%Y%m%d') + relativedelta(months=i)).strftime('%Y%m%d'))
        if observeDate != TechVal.unKnowTradeDate:
            observeList.append(observeDate)
        else:
            isObserveListComplete = False
    payType,payDate,payOff,isKnockIn,isKnockOut,knockInList,knockOutList = simulate(anchorDate,anchorPrice,knockOutPrice,knockInPrice,observeList)        
    appendList = []
    # 锚定日期
    appendList.append(str(anchorDate))
    # 锚定价格
    appendList.append('%.4f'%anchorPrice)
    # 敲入价格
    appendList.append('%.4f'%knockInPrice)
    # 敲出价格
    appendList.append('%.4f'%knockOutPrice)
    # 观察日列表
    appendList.append(','.join(observeList))
    # 观察日列表是否完整
    if isObserveListComplete:
        appendList.append(str(isObserveListComplete))
    else:
        appendList.append(str(isObserveListComplete))
    # 结束类型
    appendList.append(str(payType))
    # 结束日期
    appendList.append(str(payDate))
    # 本金和收益的和
    if payOff == '':
        appendList.append(str(payOff))
    else:
        appendList.append('%.4f'%payOff)
    # 是否敲入
    appendList.append((str(isKnockIn)))
    # 是否敲出
    appendList.append((str(isKnockOut)))
    # 敲入信息
    appendList.append(','.join(knockInList))
    # 敲出信息
    appendList.append(','.join(knockOutList))
    # 写入sheet页
    wsSim.append(appendList)
    
fileName = BusinessVal.underlyingAsset + ' ' + '雪球模拟-' + datetime.datetime.now().strftime('%Y%m%d%H%M%S') + '.xlsx'
print(fileName)
wb.save(fileName)
quote.to_excel(BusinessVal.underlyingAsset + ' ' + '雪球行情-' + datetime.datetime.now().strftime('%Y%m%d%H%M%S') + '.xlsx')

print('FINISH')

20100104
20100105
20100106
20100107
20100108
20100111
20100112
20100113
20100114
20100115
20100118
20100119
20100120
20100121
20100122
20100125
20100126
20100127
20100128
20100129
20100201
20100202
20100203
20100204
20100205
20100208
20100209
20100210
20100211
20100212
20100222
20100223
20100224
20100225
20100226
20100301
20100302
20100303
20100304
20100305
20100308
20100309
20100310
20100311
20100312
20100315
20100316
20100317
20100318
20100319
20100322
20100323
20100324
20100325
20100326
20100329
20100330
20100331
20100401
20100402
20100406
20100407
20100408
20100409
20100412
20100413
20100414
20100415
20100416
20100419
20100420
20100421
20100422
20100423
20100426
20100427
20100428
20100429
20100430
20100504
20100505
20100506
20100507
20100510
20100511
20100512
20100513
20100514
20100517
20100518
20100519
20100520
20100521
20100524
20100525
20100526
20100527
20100528
20100531
20100601
20100602
20100603
20100604
20100607
20100608
20100609
20100610
20100611
20100617
20100618
20100621
2